Bonjour bienvenue dans le CPP solver

In [1]:
import cppyy
import numpy as np
import ctypes
from bqplot import Graph, LinearScale, ColorScale, Figure, Tooltip
from ipywidgets import HBox,Label,IntSlider,Button,Layout,interact,dlink,interactive, fixed
from string import ascii_uppercase

import ipywidgets as widgets

(Re-)building pre-compiled headers (options: -O2 -mavx); this may take a minute ...


/home/damien/miniconda3/envs/env1/lib/python3.9/site-packages/cppyy_backend/loader.py:113: UserWarning: No precompiled header available (failed to build); this may impact performance.
  warnings.warn('No precompiled header available (%s); this may impact performance.' % msg)


AttributeError: <namespace cppyy.gbl at 0x7ffff1a1b370> has no attribute 'gSystem'. Full details:
  type object '' has no attribute 'gSystem'
  'gSystem' is not a known C++ class
  'gSystem' is not a known C++ template
  'gSystem' is not a known C++ enum

In [2]:
!make clean

rm *.so
rm *.o


In [3]:
!make

g++ -o Arete.o -fPIC -c Arete.cpp
g++ -o Sommet.o -fPIC -c Sommet.cpp
g++ -o Graphe.o -fPIC -c Graphe.cpp
g++ -o Tuyau.o -fPIC -c Tuyau.cpp
g++ -o FordFulkerson.o -fPIC -c FordFulkerson.cpp
g++ -o CreerGraphe.o -fPIC -c CreerGraphe.cpp
g++ -o Dijkstra.o -fPIC -c Dijkstra.cpp
g++ -o BellmanFord.o -fPIC -c BellmanFord.cpp
g++ -o FloydWarshall.o -fPIC -c FloydWarshall.cpp
g++ -o Dmain.o -fPIC -c Dmain.cpp
g++ -fPIC -shared -o libGraph.so Arete.o Sommet.o Graphe.o Tuyau.o FordFulkerson.o CreerGraphe.o Dijkstra.o BellmanFord.o FloydWarshall.o Dmain.o


In [4]:
cppyy.include("features.h")
from Core import *

In [5]:
cppyy.load_library("libGraph")
#cppyy.load_library("libArete")
#cppyy.load_library("libSommet")
#cppyy.load_library("libGraphe")
#cppyy.load_library("libTuyau")
#cppyy.load_library("libFordFulkerson")
#cppyy.load_library("libCreerGraphe")

#cppyy.load_library("libDijkstra")
#cppyy.load_library("libBellmanFord")
#cppyy.load_library("libFloydWarshall")
#cppyy.load_library("libDmain")

In [6]:
g = cppyy.gbl.createGraphe()
algo = createAlgoPcc("BellmanFord", g)
som = createNodesPCC(g)
ar = createEdgesPCC(g)

def maj(x):
    
    ######## SAVE ########
    node_data_save = som
    colors_save = colors
    link_data_save = ar
    
    ####### CHANGE ########
    graph.colors = ['black']
    graph.node_data = list('XXXXXXX')
    graph.link_data = [{'source': 1, 'target': 1}]
    
    ####### MODIF #########
    creationColors = ['white'] * len(g.getSommets())
    creationColors[algo.getHistorique()[x].getSommetCible().getNum()] = 'green'
    creationColors[algo.getHistorique()[x].getSommetActuel().getNum()] = 'red'
    
    
    ####### RETABLIR ########
    graph.colors = creationColors
    graph.node_data = node_data_save
    graph.link_data = link_data_save
    
    displayCommentPCC(algo, x, "BellmanFord", som)

    
fig_layout = Layout(width='960px', height='500px')
colors = ['white']

graph = Graph(node_data=som, highlight_links = True, link_data=ar, link_type='line', charge=-600, colors=['white'])

interact(maj, x=widgets.IntSlider(min=0, max=len(algo.getHistorique())-1, step=1, value=0))
Figure(marks=[graph], layout=fig_layout)

interactive(children=(IntSlider(value=0, description='x', max=37), Output()), _dom_classes=('widget-interact',…

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='960…

In [7]:
cx = []
cy = []

def print_event(self, target):
    global cx
    global cy
    #print(target['data']['x'])
    print("position enregistrée")

    #{'vx' : (target['data']['x'])}
    cx.append(target['data']['x'])
    cy.append(target['data']['y'])

    
graph.selected
graph.on_element_click(print_event)

In [12]:
node_data = som
link_data = ar

c=[]
for i in range(len(cy)):
    global c
    c.append(500-cy[i])

if(len(cx) < len(node_data)):
    print("ATTENTION :\nN'oubliez pas d'enregistrer les positions en cliquant sur chaque sommet du graphe mobile.\nVérifiez bien qu'il est écrit 'position enregistrée' en dessous du graphe lors de chaque clic.\nVous devez avoir autant de message que de sommets sur le graphe mobile\nRéactualisez ensuite cette cellule. Merci.")
elif(len(cx) > len(node_data)):
    print("ATTENTION :\nTrop de sommets ont été sélectionnés.\nVeuillez réinitialiser la cellule précédente (shift+enter), puis resélectionner tous les sommets.\nRéactualisez ensuite cette cellule. Merci.")

graph = Graph(node_data=node_data, link_data=link_data, link_type='line',
              colors=['orange'], directed=True, 
              scales={'x': LinearScale(), 'y': LinearScale(), 'link_color': ColorScale(scheme='Greens')}, 
              x=cx, y=c, color=np.random.rand(7))

interact(maj, x=widgets.IntSlider(min=0, max=len(algo.getHistorique())-1, step=1, value=0))
Figure(marks=[graph], layout=fig_layout)

ATTENTION :
N'oubliez pas d'enregistrer les positions en cliquant sur chaque sommet du graphe mobile.
Vérifiez bien qu'il est écrit 'position enregistrée' en dessous du graphe lors de chaque clic.
Vous devez avoir autant de message que de sommets sur le graphe mobile
Réactualisez ensuite cette cellule. Merci.


interactive(children=(IntSlider(value=0, description='x', max=1), Output()), _dom_classes=('widget-interact',)…

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='960…

In [13]:
tooltip = Tooltip(fields=['label', 'foo'], formats=['', '', ''])
graph.tooltip = tooltip